In [ ]:
# setup

%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
from wine_analysis_hplc_uv import definitions
from wine_analysis_hplc_uv.signal_processing.mindex_signal_processing import SignalProcessor
from pybaselines import Baseline
import matplotlib.pyplot as plt

scipro = SignalProcessor()
df = pd.read_parquet(definitions.TEST_PARQ_PATH)
df

## Universal Time Axis

First lets get rid of the index column and set mins to the index:


In [ ]:
df1 = (
    df.rename({"torbreck-struie": "333"}, axis=1, level="samplecode")
    .stack(["samplecode", "wine"])
    .reset_index()
    .drop("i", axis=1)
    .set_index(["samplecode", "wine", "mins"])
    .sort_index()
    .unstack(["samplecode", "wine"])
    .reorder_levels(["samplecode", "wine", "vars"], axis=1)
)
df1

## Zeroing the Offset

Next, just like we subtracted x[0] from x, we should subtract y[0] from y so that the initial value is 0. At time zero, y = 0.


In [ ]:
df2 = (
    df1
    .pipe(scipro.zero_y_axis)
    .pipe(lambda df: df if df.pipe(scipro.vars_subplots) else df)
)
plt.suptitle("zeroed")
# df2

## Min-max Scaling

In [ ]:


df3 = (
    df2
    .pipe(scipro.min_max_scale)
    .pipe(lambda df: df if df.pipe(scipro.vars_subplots) else df)
)
plt.suptitle("min-maxed")

## Subsetting x-Axis

Now we dont want the baseline to be fitting the dead area after ~ 25 mins, but we need a way of identifying where the curve flattens out, i.e. the first derivative. While I would like to spend time working on this, I need to push on. we'll call an arbritary limit of 20 mins.
TODO: look into interpretatoin of first and second derivatives


In [ ]:
df4 = df3.loc[df3.index < pd.to_timedelta(20, unit="minutes")]  # drop data after 20 min
df4

## Write to file

Now that the y-axis has been zeroed, scaled and subset, time to write to parquet file for downstream processing.

In [ ]:
df4.to_parquet(definitions.PRO_PARQ_PATH)